In [4]:
# input: txt edges (not adj list, 
#                   requir the tail of each edge 
#                   to be sorted)
# output: dict graph
# sink vertices are followed by empty lists
def preprocess_edge_list(filename):
    edge_list = []
    with open(filename) as f_handle:
        for line in f_handle:
            edge_list.append(line.split())  # vertices exist as str
            
    #print(edge_list)
    return adj_list(edge_list)


# input: edge list
# output: dict graph
def adj_list(edge_list):

    num_v = int(edge_list[-1][0])  # the tails of edges follow numerical order
    vertices = [i for i in range(1, num_v + 1)]
    graph_obj = dict(list((i, []) for i in vertices))    # sink vertex is indicated by the value []
    
    for i, j in edge_list:  # i, j are str
        graph_obj[int(i)].append(int(j))
    
    #print(graph_obj)
    return (graph_obj)


# define vertex, Graph classes
# Vertex class for directed graphs (object with 'key', 'tail_of', and 'head_of' keys)

# remark: In particular, a directed edge is specified as an ordered pair of vertices u, v
# and is denoted by (u, v) or u -> v. In this case, u is the tail of the edge and v is the
# head
class Vertex():
    def __init__(self, key):
        self._key = key
        self._tail_of = {} # where to go
        self._head_of = {} # how to come

    def __str__(self):
        return '{' + "'key': {}, 'tail_of': {}, 'head_of': {}".format(
            self._key,
            self._tail_of,
            self._head_of
        ) + '}'

    def add_head(self, head_key, weight=1):
        if (head_key):
            self._tail_of[head_key] = weight

    def add_tail(self, tail_key, weight=1):
        if (tail_key):
            self._head_of[tail_key] = weight

    def tail_of(self, head_key):
        return head_key in self._tail_of

    def head_of(self, tail_key):
        return tail_key in self._head_of

    def get_tail_of_keys(self):
        return list(self._tail_of.keys())

    def get_head_of_keys(self):
        return list(self._head_of.keys())

    def remove_tail(self, tail_key):
        if tail_key in self._head_of:
            del self._head_of[tail_key]

    def remove_head(self, head_key):
        if head_key in self._tail_of:
            del self._tail_of[head_key]

    def get_tail_weight(self, tail_key):
        if tail_key in self._head_of:
            return self._head_of[tail_key]

    def get_head_weight(self, head_key):
        if head_key in self._tail_of:
            return self._tail_of[head_key]


# Directed graph class
class Graph():
    def __init__(self):
        self._vertices = {}

    # 'x in graph' will use this containment logic
    def __contains__(self, key):
        return key in self._vertices

    # 'for x in graph' will use this iter() definition, where x is a vertex in an array
    def __iter__(self):
        return iter(self._vertices.values())

    def __str__(self):
        output = '\n{\n'
        
        vertices = self._vertices.values()
        for v in vertices:
            graph_key = "{}".format(v._key)
            v_str = "\n   'key': {}, \n   'tail_of': {}, \n   'head_of': {}".format(
                v._key,
                v._tail_of,
                v._head_of
            )
            output += ' ' + graph_key + ': {' + v_str + '\n },\n'
        return output + '}'

    def add_v(self, v):
        if v:
            self._vertices[v._key] = v
        return self

    def get_v(self, key):
        try:
            return self._vertices[key]
        except KeyError:
            return None

    def get_v_keys(self):
        return list(self._vertices.keys())

    # removes vertex as head and tail from all its neighbors, then deletes vertex
    def remove_v(self, key):
        if key in self._vertices:
            head_of_keys = self._vertices[key].get_head_of_keys()
            tail_of_keys = self._vertices[key].get_tail_of_keys()
            for tail_key in head_of_keys:
                self.remove_head(tail_key, key)
            for head_key in tail_of_keys:
                self.remove_tail(key, head_key)
            del self._vertices[key]

    def add_e(self, tail_key, head_key, weight=1):
        if tail_key not in self._vertices:
            self.add_v(Vertex(tail_key))
        if head_key not in self._vertices:
            self.add_v(Vertex(head_key))

        self._vertices[tail_key].add_head(head_key, weight)

    def get_e(self, tail_key, head_key):
        if tail_key and head_key in self._vertices:
            return self.get_v(tail_key).get_e(head_key)

    # adds the weight for an edge if it exists already, with a default of 1
    def increase_e(self, tail_key, head_key, weight=1):
        if tail_key not in self._vertices:
            self.add_v(Vertex(tail_key))
        if head_key not in self._vertices:
            self.add_v(Vertex(head_key))

        w_v1_v2 = self.get_v(tail_key).get_head_weight(head_key)
        new_w_v1_v2 = w_v1_v2 + weight if w_v1_v2 else weight

        self._vertices[tail_key].add_head(head_key, new_w_v1_v2)

    def has_forward_e(self, tail_key, head_key):
        if tail_key in self._vertices:
            return self._vertices[tail_key].tail_of(head_key)

    def remove_tail(self, tail_key, head_key):
        if head_key in self._vertices:
            self._vertices[head_key].remove_tail(tail_key)

    def remove_head(self, tail_key, head_key):
        if tail_key in self._vertices:
            self._vertices[tail_key].remove_head(head_key)

    def remove_e(self, tail_key, head_key):
        if tail_key in self._vertices:
            self._vertices[tail_key].remove_head(head_key)
        if head_key in self._vertices:
            self._vertices[head_key].remove_tail(tail_key)

    def for_each_v(self, cb):
        for v in self._vertices:
            cb(v)
            
            
# input: adj_list(graph_object)
# output: Graph
def creat_graph(graph_obj):
    G = Graph()
    for v_key in graph_obj:  # equavilant of for v_key in vertices
        
        # establish vertex v for adding heads and tails
        # the 'else' condition is to capture already processed vertices
        v = Vertex(v_key) if v_key not in G else G.get_v(v_key)
        
        # v_key -> head_key
        for head_key in graph_obj[v_key]:
            
            # forward
            v.add_head(head_key)
            
            # reversed
            v_head = Vertex(head_key) if head_key not in G else G.get_v(head_key)
            v_head.add_tail(v_key)
            
            # save changes
            G.add_v(v_head)
            
        G.add_v(v)
        
    return(G)


from collections import deque as Stack

# input: G
# output: TO (or sudo-TO for looping structure) in node's names
# Theorem: Every directed acyclic graph 
#          has at least one topological ordering

def TS(G, reversed = False, order = None):
    global E, currL, TO
    
    # keeps track of ordering in node's names
    # so we don't have to sort by TO
    TO = Stack()
    if not order:
        if reversed:
            #nodes = sorted(list(G.get_v_keys()), reverse=True)
            nodes = list(G.get_v_keys())
        else:
            #nodes = sorted(list(G.get_v_keys()))
            nodes = list(G.get_v_keys())
    else:
        nodes = order
    currL = len(nodes)
    # mark all vertices as unexplored
    E = dict(list(zip(nodes, [0]*len(nodes))), key=lambda x: x[0])
    
    for v in nodes:
        if not E[v]:
            DFS_TS_rec(G, v, reversed=reversed)
            
    return(TO)

def DFS_TS_rec(G, s, reversed):
    global E, currL, TO
    E[s] = 1
    if reversed:
        for v in G.get_v(s).get_head_of_keys():
            if not E[v]:
                DFS_TS_rec(G, v, reversed=reversed)
    else:
        for v in G.get_v(s).get_tail_of_keys():
            if not E[v]:
                DFS_TS_rec(G, v, reversed=reversed)
    
    TO.appendleft(s)


def DFS_SCC_rec(G, s):
    global E, SCCs, numSCC
    E[s] = 1
    SCCs[s] = numSCC
    for v in G.get_v(s).get_tail_of_keys():
        if not E[v]:
            DFS_SCC_rec(G, v)
            
            
def SCC(G):
    global numSCC, E, SCCs
    numSCC = 0
    
    TO = TS(G, reversed=True)
    E = dict(list(zip(TO, [0]*len(TO))), key=lambda x: x[0])
    SCCs = E.copy()
    
    # in increasing order of f(v) from G_rev
    for v in TO:
        if not E[v]:
            numSCC += 1
            DFS_SCC_rec(G, v)
            
    return(SCCs)


def find_largest(SCCs):
    import pandas as pd
    return(list(pd.Series(SCCs.values()).value_counts(sort=True).values[:-1]) + 
           [0] * (0 if ((5-len(list(pd.Series(SCCs.values()).value_counts(sort=True).values[:-1]))) < 0) 
                  else (5-len(list(pd.Series(SCCs.values()).value_counts(sort=True).values[:-1])))))

In [5]:
import sys, threading, time
sys.setrecursionlimit(800000)
threading.stack_size(67108864)

def main():
    
    graph_obj = preprocess_edge_list('t.txt')
    G = creat_graph(graph_obj)
    start = time.time()
    SCCs = SCC(G)
    print (find_largest(SCCs)[:5])
    print('time elapse: {}'.format(time.time() - start))

In [6]:
thread = threading.Thread(target=main)
thread.start()

[3, 3, 1, 1, 0]
time elapse: 0.008208990097045898


main()

graph_obj = preprocess_edge_list('t.txt')
G = creat_graph(graph_obj)
TS(G)